# FINAL UPDATE: this is not a problem, the assaylayout can exclude certain FOVs

In [1]:
import enum
import os
import re
from typing import Tuple

import numpy as np
from scipy.ndimage import median_filter

import xml.etree.ElementTree as ET
from tqdm.auto import tqdm
import pandas as pd

In [3]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'

In [2]:
def read_harmony_metadata(metadata_path: os.PathLike, assay_layout = False, 
                          mask_layout = False
    )-> pd.DataFrame:
    """
    Read the metadata from the Harmony software for the Opera Phenix microscope.
    Takes an input of the path to the metadata .xml file.
    Returns the metadata in a pandas dataframe format.
    If assay_layout is True then alternate xml format is anticipated, returning
    information about the assay layout of the experiment rather than the general
    organisation of image volume.
    """
    ### read xml metadata file
    print('Reading metadata XML file...')
    xml_data = open(metadata_path, 'r', encoding="utf-8-sig").read()
    root = ET.XML(xml_data)
    ### extraction procedure for image volume metadata
    if not assay_layout:
        ### extract the metadata from the xml file
        images_metadata = [child for child in root if "Images" in child.tag][0]
        ### create an empty list for storing individual image metadata
        metadata = list()
        ### iterate over every image entry extracting the metadata
        for image_metadata in tqdm(images_metadata, total = len(images_metadata),
                                    desc = 'Extracting HarmonyV5 metadata'):
            ### create empty dict to store single image metadata
            single_image_dict = dict()
            ### iterate over every metadata item in that image metadata
            for item in image_metadata:
                ### get column names from metadata
                col = item.tag.replace('{http://www.perkinelmer.com/PEHH/HarmonyV5}','')
                ### get metadata
                entry = item.text
                ### make dictionary out of metadata
                single_image_dict[col] = entry
            ### append that image metadata to list of all images
            metadata.append(single_image_dict)
    ### extraction procedure for assay layout metadata
    if assay_layout:
        metadata = dict()
        for branch in root:
            for subbranch in branch:
                if subbranch.text.strip() and subbranch.text.strip() != 'string':
                    col_name = subbranch.text
                    metadata[col_name] = dict()
                for subsubbranch in subbranch:
                    if 'Row' in subsubbranch.tag:
                        row = int(subsubbranch.text)
                    elif 'Col' in subsubbranch.tag and 'Color' not in subsubbranch.tag:
                        col = int(subsubbranch.text)
                    if 'Value' in subsubbranch.tag and subsubbranch.text != None:
                        val = subsubbranch.text
                        metadata[col_name][int(row), int(col)] = val

    ### create a dataframe out of all metadata
    df = pd.DataFrame(metadata)

    if assay_layout and mask_exist:
        df['Missing masks'] = np.nan
        for index, row in df.iterrows():
            row, col = index
            missing_mask_dict = do_masks_exist(metadata, row = row, col = col, print_output = False)
            df.at[(row, col), 'Missing masks'] = missing_mask_dict[row, col]

    print('Extracting metadata complete!')
    return df

In [19]:
metadata = read_harmony_metadata(metadata_path)

Reading metadata XML file...


IndexError: list index out of range

In [9]:
assay_layout = True
mask_exist = False

In [10]:
### read xml metadata file
print('Reading metadata XML file...')
xml_data = open(metadata_path, 'r', encoding="utf-8-sig").read()
root = ET.XML(xml_data)
### extraction procedure for image volume metadata
if not assay_layout:
    ### extract the metadata from the xml file
    images_metadata = [child for child in root if "Images" in child.tag][0]
    ### create an empty list for storing individual image metadata
    metadata = list()
    ### iterate over every image entry extracting the metadata
    for image_metadata in tqdm(images_metadata, total = len(images_metadata),
                                desc = 'Extracting HarmonyV5 metadata'):
        ### create empty dict to store single image metadata
        single_image_dict = dict()
        ### iterate over every metadata item in that image metadata
        for item in image_metadata:
            ### get column names from metadata
            col = item.tag.replace('{http://www.perkinelmer.com/PEHH/HarmonyV5}','')
            ### get metadata
            entry = item.text
            ### make dictionary out of metadata
            single_image_dict[col] = entry
        ### append that image metadata to list of all images
        metadata.append(single_image_dict)
### extraction procedure for assay layout metadata
if assay_layout:
    metadata = dict()
    for branch in root:
        for subbranch in branch:
            if subbranch.text.strip() and subbranch.text.strip() != 'string':
                col_name = subbranch.text
                metadata[col_name] = dict()
            for subsubbranch in subbranch:
                if 'Row' in subsubbranch.tag:
                    row = int(subsubbranch.text)
                elif 'Col' in subsubbranch.tag and 'Color' not in subsubbranch.tag:
                    col = int(subsubbranch.text)
                if 'Value' in subsubbranch.tag and subsubbranch.text != None:
                    val = subsubbranch.text
                    metadata[col_name][int(row), int(col)] = val

### create a dataframe out of all metadata
df = pd.DataFrame(metadata)

if assay_layout and mask_exist:
    df['Missing masks'] = np.nan
    for index, row in df.iterrows():
        row, col = index
        missing_mask_dict = do_masks_exist(metadata, row = row, col = col, print_output = False)
        df.at[(row, col), 'Missing masks'] = missing_mask_dict[row, col]

print('Extracting metadata complete!')


Reading metadata XML file...
Extracting metadata complete!


In [17]:
df

Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

In [29]:
len(df)

24

# Missing column 3 for every row

or is this just the wrong assay layout file for the experiment?

In [16]:
metadata = dict()
for branch in root:
    for subbranch in branch:
        if subbranch.text.strip() and subbranch.text.strip() != 'string':
            col_name = subbranch.text
            metadata[col_name] = dict()
        for subsubbranch in subbranch:
            if 'Row' in subsubbranch.tag:
#                 print(row)
                row = int(subsubbranch.text)
            elif 'Col' in subsubbranch.tag and 'Color' not in subsubbranch.tag:
                col = int(subsubbranch.text)
#                 print(col)
            if 'Value' in subsubbranch.tag and subsubbranch.text != None:
                val = subsubbranch.text
                metadata[col_name][int(row), int(col)] = val
                print(row, col, col_name, val)
### create a dataframe out of all metadata
df = pd.DataFrame(metadata)


3 4 Strain RD1
3 5 Strain WT
3 6 Strain WT
3 7 Strain WT
3 8 Strain WT
3 9 Strain WT
4 4 Strain RD1
4 5 Strain WT
4 6 Strain WT
4 7 Strain WT
4 8 Strain WT
4 9 Strain WT
5 4 Strain RD1
5 5 Strain WT
5 6 Strain WT
5 7 Strain WT
5 8 Strain WT
5 9 Strain WT
6 4 Strain RD1
6 5 Strain WT
6 6 Strain WT
6 7 Strain WT
6 8 Strain WT
6 9 Strain WT
3 4 Compound CTRL
3 5 Compound CTRL
3 6 Compound PZA
3 7 Compound RIF
3 8 Compound INH
3 9 Compound BDQ
4 4 Compound CTRL
4 5 Compound CTRL
4 6 Compound PZA
4 7 Compound RIF
4 8 Compound INH
4 9 Compound BDQ
5 4 Compound CTRL
5 5 Compound CTRL
5 6 Compound PZA
5 7 Compound RIF
5 8 Compound INH
5 9 Compound BDQ
6 4 Compound CTRL
6 5 Compound CTRL
6 6 Compound PZA
6 7 Compound RIF
6 8 Compound INH
6 9 Compound BDQ
3 4 Concentration 0
3 5 Concentration 0
3 6 Concentration 60
3 7 Concentration 0.1
3 8 Concentration 0.04
3 9 Concentration 0.02
4 4 Concentration 0
4 5 Concentration 0
4 6 Concentration 60
4 7 Concentration 0.1
4 8 Concentration 0.04
4 9 Conce

In [23]:
for branch in root:
    print(branch)
    for subbranch in branch:
        #print(subbranch.text)
        if subbranch.text.strip() and subbranch.text.strip() != 'string':
            col_name = subbranch.text
            metadata[col_name] = dict()
        for subsubbranch in subbranch:
            if 'Row' in subsubbranch.tag:
#                 print(row)
                row = int(subsubbranch.text)
            elif 'Col' in subsubbranch.tag and 'Color' not in subsubbranch.tag:
                col = int(subsubbranch.text)
#                 print(col)
            if 'Value' in subsubbranch.tag and subsubbranch.text != None:
                val = subsubbranch.text
                metadata[col_name][int(row), int(col)] = val
                #print(row, col, col_name, val)

<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}PlateCols' at 0x7f4bf0765a40>
<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}PlateRows' at 0x7f4bf0765a90>
<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}Layer' at 0x7f4bf0765b30>
<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}Layer' at 0x7f4bf0961720>
<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}Layer' at 0x7f4bf097b1d0>
<Element '{http://www.perkinelmer.com/PEHH/HarmonyV5}Layer' at 0x7f4bf0813c20>


In [28]:
branch.findtext

Help on built-in function findtext:

findtext(path, default=None, namespaces=None) method of xml.etree.ElementTree.Element instance

